In [1]:
%%bash
date

Thu Jun 22 14:26:20 PDT 2017


In [2]:
%%bash
system_profiler SPSoftwareDataType

Software:

    System Software Overview:

      System Version: Mac OS X 10.7.5 (11G63)
      Kernel Version: Darwin 11.4.2
      Boot Volume: SSD2
      Boot Mode: Normal
      Computer Name: greenbird (2)
      User Name: Sam (Sam)
      Secure Virtual Memory: Enabled
      64-bit Kernel and Extensions: No
      Time since boot: 38 days 3:04



In [3]:
%%bash
#Uses grep to exclude lines that display serial number and hardware UUID
system_profiler SPHardwareDataType | grep -v [SH][ea]



      Model Name: Mac Pro
      Model Identifier: MacPro1,1
      Processor Name: Dual-Core Intel Xeon
      Processor Speed: 3 GHz
      Number of Processors: 2
      Total Number of Cores: 4
      L2 Cache (per Processor): 4 MB
      Memory: 14 GB
      Bus Speed: 1.33 GHz
      Boot ROM Version: MP11.005C.B08
      SMC Version (system): 1.7f10



```
Create md5 checksums of original files
```

```
XML array: xml_array=($(find . -maxdepth 2 -name "*.xml"))
fastq array: fastq_array=($(find . -maxdepth 2 -name "*.fastq.gz"))
rm path: xml_nopath_array=($(echo "${array[@]##*/}"))
rm suffix: xml_nosuffix_array=($(echo "${array3[@]%%.*}"))
renaming loop:
for item in "${!fastq_array[@]}"; do
fastq_name=${fastq_array[$item]}
xml_name=${xml_nosuffix_array[$item]}
cp fastq_name /owl/nightingales/O_lurida/"${xml_name}_${fastq_name}"
```